1) Unit тесты (тесты для функций создания игры, коммита и раскрытия хода, определения победителя)

In [ ]:
const { expect } = require("chai");

describe("RockPaperScissors", function () {
    let RockPaperScissors, rps, player1, player2;

    // Перед каждым тестом разворачиваем новый контракт и получаем ссылки на тестовые аккаунты
    beforeEach(async function () {
        RockPaperScissors = await ethers.getContractFactory("RockPaperScissors"); // Получаем контракт для деплоя
        [player1, player2] = await ethers.getSigners(); // Получаем два аккаунта для тестирования
        rps = await RockPaperScissors.deploy(); // Разворачиваем контракт
        await rps.deployed(); // Дожидаемся завершения деплоя
    });

    // Тестируем создание новой игры
    it("Should create a game", async function () {
        const wager = ethers.utils.parseEther("1"); // Устанавливаем ставку в 1 ETH
        await rps.connect(player1).createGame(player2.address, { value: wager }); // Создаем игру от имени player1 с передачей ставки

        const game = await rps.games(1); // Получаем данные о созданной игре

        // Проверяем, что данные игры соответствуют ожидаемым значениям
        expect(game.player1).to.equal(player1.address);
        expect(game.player2).to.equal(player2.address);
        expect(game.wager).to.equal(wager);
        expect(game.isActive).to.be.true;
    });

    // Тестируем коммит хода
    it("Should commit a move", async function () {
        const wager = ethers.utils.parseEther("1"); // Устанавливаем ставку
        await rps.connect(player1).createGame(player2.address, { value: wager }); // Создаем игру

        // Создаем хеш от хода и "соли" (секретного значения)
        const commitment = ethers.utils.keccak256(ethers.utils.toUtf8Bytes("Rock|secret"));

        await rps.connect(player1).commitMove(1, commitment); // Игрок 1 делает коммит хода
        const game = await rps.games(1); // Получаем данные о игре

        expect(game.player1Commit).to.equal(commitment); // Проверяем, что коммит записан
    });

    // Тестируем раскрытие хода и определение победителя
    it("Should reveal a move and determine a winner", async function () {
        const wager = ethers.utils.parseEther("1"); // Устанавливаем ставку
        await rps.connect(player1).createGame(player2.address, { value: wager }); // Создаем игру
        
        // Создаем коммиты для двух игроков
        const commitment1 = ethers.utils.keccak256(ethers.utils.toUtf8Bytes("Rock|secret1"));
        const commitment2 = ethers.utils.keccak256(ethers.utils.toUtf8Bytes("Scissors|secret2"));
        
        // Игроки делают коммиты своих ходов
        await rps.connect(player1).commitMove(1, commitment1);
        await rps.connect(player2).commitMove(1, commitment2);

        // Игроки раскрывают свои ходы
        await rps.connect(player1).revealMove(1, 1, "secret1"); // Игрок 1 раскрывает "Камень"
        await rps.connect(player2).revealMove(1, 3, "secret2"); // Игрок 2 раскрывает "Ножницы"

        const game = await rps.games(1);
        expect(game.isActive).to.be.false; // Проверяем, что игра завершена
    });
});


2) Вспомогательный контракт для вызова метода из RockPaperScissors

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

// Интерфейс для взаимодействия с контрактом RockPaperScissors
interface IRockPaperScissors {
    function createGame(address opponent) external payable;
}

contract GameStarter {
    IRockPaperScissors public rps;

    // Конструктор получает адрес основного контракта RockPaperScissors
    constructor(address _rpsAddress) {
        rps = IRockPaperScissors(_rpsAddress); // Сохраняем ссылку на основной контракт
    }

    // Функция для создания новой игры через основной контракт
    function startNewGame(address opponent) external payable {
        require(msg.value > 0, "Wager must be greater than 0."); // Проверяем, что ставка больше 0
        rps.createGame{value: msg.value}(opponent); // Вызываем функцию создания игры в основном контракте
    }
}


3) Интеграционные тесты для межконтрактного взаимодействия

In [ ]:
describe("GameStarter with RockPaperScissors", function () {
    let RockPaperScissors, GameStarter, rps, gameStarter, player1, player2;

    // Перед каждым тестом развернем новые экземпляры контрактов и получим аккаунты
    beforeEach(async function () {
        RockPaperScissors = await ethers.getContractFactory("RockPaperScissors");
        GameStarter = await ethers.getContractFactory("GameStarter");
        [player1, player2] = await ethers.getSigners();

        rps = await RockPaperScissors.deploy(); // Деплой основного контракта
        await rps.deployed();

        gameStarter = await GameStarter.deploy(rps.address); // Деплой вспомогательного контракта
        await gameStarter.deployed();
    });

    // Тестируем создание игры через вспомогательный контракт GameStarter
    it("Should start a new game via GameStarter", async function () {
        const wager = ethers.utils.parseEther("1"); // Устанавливаем ставку
        await gameStarter.connect(player1).startNewGame(player2.address, { value: wager }); // Создаем игру через вспомогательный контракт

        const game = await rps.games(1); // Проверяем данные о созданной игре в основном контракте

        // Проверяем, что данные совпадают с ожидаемыми
        expect(game.player1).to.equal(player1.address);
        expect(game.player2).to.equal(player2.address);
        expect(game.wager).to.equal(wager);
    });
});


4) Актуальное состояние индустрии в области создания формальных спецификаций и проведения формальных проверок  смарт-контрактов

Формальные спецификации

Формальная спецификация — это точное математическое описание логики программы. Для смарт-контрактов, работающих с цифровыми активами, она помогает минимизировать ошибки и повысить безопасность. Основные инструменты для создания формальных спецификаций включают:

TLA+: язык для описания и моделирования параллельных и распределенных систем, применяемый к сложной логике смарт-контрактов.
Coq и Isabelle: интерактивные доказательные системы для написания спецификаций и их доказательств, гарантируя соответствие кода строгим требованиям.
K Framework: инструмент для создания формальных спецификаций языков программирования и анализа семантики, часто используемый для языка Solidity.

Формальная верификация

Формальная верификация — процесс математического доказательства соответствия программы спецификации. Для смарт-контрактов это включает проверку безопасности и отсутствие уязвимостей, таких как переполнения или атаки с повторным вызовом (reentrancy). Основные инструменты:

SMTChecker: встроен в компилятор Solidity, использует теории и ограничения для автоматического доказательства свойств, например, защиты от переполнений.
Certora Prover: коммерческий инструмент для проверки пользовательских условий и свойств, применяемый для верификации логики управления и токенов ERC20.
MythX и Slither: инструменты статического анализа, находящие типичные ошибки и уязвимости.
VerX: инструмент для проверки временных свойств смарт-контрактов, позволяющий проверять последовательность выполнения условий и операций.
Сложности и задачи
Формальная верификация требует глубоких знаний и значительных усилий. Ключевые сложности:

Сложность математики и логики: написание спецификаций и доказательств требует знаний в области формальных методов.
Трудоемкость: процесс требует времени, особенно при создании доказательств для крупных систем.
Изменяющаяся экосистема: развитие Solidity и блокчейна требует регулярной адаптации формальных методов.